# Getting Started with LlamaStack: Tool Calling Tutorial

Welcome! This notebook will guide you through creating and using custom tools with LlamaStack.
We'll start with the basics and work our way up to more complex examples.

Table of Contents:
1. Setup and Installation
2. Understanding Tool Basics
3. Creating Your First Tool
4. Building a Mock Weather Tool
5. Setting Up the LlamaStack Agent
6. Running Examples
7. Next Steps


## 1. Setup
#### Before we begin, let's import all the required packages:

In [1]:
import os
import asyncio
import json
from typing import Dict
from datetime import datetime

In [2]:
# LlamaStack specific imports
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types.agent_create_params import AgentConfig
from llama_stack_client.types.tool_param_definition_param import ToolParamDefinitionParam

## 2. Understanding Tool Basics

In LlamaStack, a tool is like a special function that our AI assistant can use. Think of it as giving the AI a new 
capability, like using a calculator or checking the weather.

Every tool needs:
- A name: What we call the tool
- A description: What the tool does
- Parameters: What information the tool needs to work
- Implementation: The actual code that does the work

Let's create a base class that all our tools will inherit from:

In [26]:
class SingleMessageCustomTool:
    """Base class for all our custom tools"""
    
    async def run(self, messages=None):
        """
        Main entry point for running the tool
        Args:
            messages: List of messages (can be None for backward compatibility)
        """
        if messages and len(messages) > 0:
            # Extract parameters from the message if it contains function parameters
            message = messages[0]
            if hasattr(message, 'function_parameters'):
                return await self.run_impl(**message.function_parameters)
            else:
                return await self.run_impl()
        return await self.run_impl()
    
    async def run_impl(self, **kwargs):
        """Each tool will implement this method with their specific logic"""
        raise NotImplementedError()

## 3. Creating Your First Tool: Calculator
 
Let's create a simple calculator tool. This will help us understand the basic structure of a tool.
Our calculator can:
- Add
- Subtract
- Multiply
- Divide


In [27]:
# Calculator Tool implementation
class CalculatorTool(SingleMessageCustomTool):
    """A simple calculator tool that can perform basic math operations"""
    
    def get_name(self) -> str:
        return "calculator"
    
    def get_description(self) -> str:
        return "Perform basic arithmetic operations (add, subtract, multiply, divide)"
    
    def get_params_definition(self) -> Dict[str, ToolParamDefinitionParam]:
        return {
            "operation": ToolParamDefinitionParam(
                param_type="str",
                description="Operation to perform (add, subtract, multiply, divide)",
                required=True
            ),
            "x": ToolParamDefinitionParam(
                param_type="float",
                description="First number",
                required=True
            ),
            "y": ToolParamDefinitionParam(
                param_type="float",
                description="Second number",
                required=True
            )
        }
    
    async def run_impl(self, operation: str = None, x: float = None, y: float = None):
        """The actual implementation of our calculator"""
        if not all([operation, x, y]):
            return json.dumps({"error": "Missing required parameters"})
            
        # Dictionary of math operations
        operations = {
            "add": lambda a, b: a + b,
            "subtract": lambda a, b: a - b,
            "multiply": lambda a, b: a * b,
            "divide": lambda a, b: a / b if b != 0 else "Error: Division by zero"
        }
        
        # Check if the operation is valid
        if operation not in operations:
            return json.dumps({"error": f"Unknown operation '{operation}'"})
        
        try:
            # Convert string inputs to float if needed
            x = float(x) if isinstance(x, str) else x
            y = float(y) if isinstance(y, str) else y
            
            # Perform the calculation
            result = operations[operation](x, y)
            return json.dumps({"result": result})
        except ValueError:
            return json.dumps({"error": "Invalid number format"})
        except Exception as e:
            return json.dumps({"error": str(e)})

## 4. Building a Mock Weather Tool
 
Now let's create something a bit more complex: a weather tool! 
While this is just a mock version (it doesn't actually fetch real weather data),
it shows how you might structure a tool that interfaces with an external API.

In [28]:
class WeatherTool(SingleMessageCustomTool):
    "async def run_single_query(agent, session_id, query: str):
    """Run a single query through our agent with complete interaction cycle"""
    print("\n" + "="*50)
    print(f"🤔 User asks: {query}")
    print("="*50)
    
    # Get the initial response and tool call
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        session_id=session_id,
    )
    
    # Process all events including tool calls and final response
    async for event in EventLogger().log(response):
        event.print()
        
        # If this was a tool call, we need to create another turn with the result
        if hasattr(event, 'tool_calls') and event.tool_calls:
            tool_call = event.tool_calls[0]  # Get the first tool call
            
            # Execute the custom tool
            if tool_call.tool_name in [t.get_name() for t in agent.custom_tools]:
                tool = [t for t in agent.custom_tools if t.get_name() == tool_call.tool_name][0]
                result = await tool.run_impl(**tool_call.arguments)
                
                # Create a follow-up turn with the tool result
                follow_up = agent.create_turn(
                    messages=[
                        {
                            "role": "tool",
                            "content": result,
                            "tool_call_id": tool_call.call_id,
                            "name": tool_call.tool_name
                        }
                    ],
                    session_id=session_id,
                )
                
                # Process the follow-up response
                async for follow_up_event in EventLogger().log(follow_up):
                    follow_up_event.print()""A mock weather tool that simulates getting weather data"""
    
    def get_name(self) -> str:
        return "get_weather"
    
    def get_description(self) -> str:
        return "Get current weather information for major cities"
    
    def get_params_definition(self) -> Dict[str, ToolParamDefinitionParam]:
        return {
            "city": ToolParamDefinitionParam(
                param_type="str",
                description="Name of the city (e.g., New York, London, Tokyo)",
                required=True
            ),
            "date": ToolParamDefinitionParam(
                param_type="str",
                description="Date in YYYY-MM-DD format (optional)",
                required=False
            )
        }
    
    async def run_impl(self, city: str = None, date: str = None):
        if not city:
            return json.dumps({"error": "City parameter is required"})
            
        # Mock database of weather information
        weather_data = {
            "New York": {"temp": 20, "condition": "sunny"},
            "London": {"temp": 15, "condition": "rainy"},
            "Tokyo": {"temp": 25, "condition": "cloudy"}
        }
        
        try:
            # Check if we have data for the requested city
            if city not in weather_data:
                return json.dumps({
                    "error": f"Sorry! No data available for {city}",
                    "available_cities": list(weather_data.keys())
                })
            
            # Return the weather information
            return json.dumps({
                "city": city,
                "date": date or datetime.now().strftime("%Y-%m-%d"),
                "data": weather_data[city]
            })
        except Exception as e:
            return json.dumps({"error": str(e)})

In [29]:
# ## 5. Setting Up the LlamaStack Agent
# 
# Now that we have our tools, we need to create an agent that can use them.
# The agent is like a smart assistant that knows how to use our tools when needed.

In [30]:
async def setup_agent(host: str = "localhost", port: int = 5001):
    """Creates and configures our LlamaStack agent"""
    
    # Create a client to connect to the LlamaStack server
    client = LlamaStackClient(
        base_url=f"http://{host}:{port}",
    )
    
    # Configure how we want our agent to behave
    agent_config = AgentConfig(
        model="Llama3.1-8B-Instruct",
        instructions="""You are a helpful assistant that can:
        1. Perform mathematical calculations
        2. Check weather information
        Always explain your thinking before using a tool.""",
        
        sampling_params={
            "strategy": "greedy",
            "temperature": 1.0,
            "top_p": 0.9,
        },
        
        # List of tools available to the agent
        tools=[
            {
                "function_name": "calculator",
                "description": "Perform basic arithmetic operations",
                "parameters": {
                    "operation": {
                        "param_type": "str",
                        "description": "Operation to perform (add, subtract, multiply, divide)",
                        "required": True,
                    },
                    "x": {
                        "param_type": "float",
                        "description": "First number",
                        "required": True,
                    },
                    "y": {
                        "param_type": "float",
                        "description": "Second number",
                        "required": True,
                    },
                },
                "type": "function_call",
            },
            {
                "function_name": "get_weather",
                "description": "Get weather information for a given city",
                "parameters": {
                    "city": {
                        "param_type": "str",
                        "description": "Name of the city",
                        "required": True,
                    },
                    "date": {
                        "param_type": "str",
                        "description": "Date in YYYY-MM-DD format",
                        "required": False,
                    },
                },
                "type": "function_call",
            },
        ],
        tool_choice="auto",
        # Using standard JSON format for tools
        tool_prompt_format="json",  
        input_shields=[],
        output_shields=[],
        enable_session_persistence=False,
    )
    
    # Create our tools
    custom_tools = [CalculatorTool(), WeatherTool()]
    
    # Create the agent
    agent = Agent(client, agent_config, custom_tools)
    session_id = agent.create_session("tutorial-session")
    print(f"🎉 Created session_id={session_id} for Agent({agent.agent_id})")
    
    return agent, session_id

In [31]:
# ## 6. Running Examples
# 
# Let's try out our agent with some example questions!

# %%

In [46]:
import nest_asyncio
nest_asyncio.apply()  # This allows async operations to work in Jupyter

# %%
# Initialize the agent
async def init_agent():
    """Initialize our agent - run this first!"""
    agent, session_id = await setup_agent()
    print(f"✨ Agent initialized with session {session_id}")
    return agent, session_id

# %%
# Function to run a single query
async def run_single_query(agent, session_id, query: str):
    """Run a single query through our agent"""
    print("\n" + "="*50)
    print(f"🤔 User asks: {query}")
    print("="*50)
    
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        session_id=session_id,
    )
    
    async for log in EventLogger().log(response):
        log.print()

Now let's run everything and see it in action!

Create and run our agent

In [47]:
agent, session_id = await init_agent()

🎉 Created session_id=fbe83bb6-bdfd-497c-b920-d7307482d8ba for Agent(3997eeda-4ffd-4b05-9026-28b4da206a11)
✨ Agent initialized with session fbe83bb6-bdfd-497c-b920-d7307482d8ba


In [48]:
await run_single_query(agent, session_id, "What's 25 plus 17?")


🤔 User asks: What's 25 plus 17?
inference> {"type": "function", "name": "calculator", "parameters": {"operation": "add", "y": "17", "x": "25"}}


In [49]:
await run_single_query(agent, session_id, "What's the weather like in Tokyo?")


🤔 User asks: What's the weather like in Tokyo?
inference> {"type": "function", "name": "get_weather", "parameters": {"city": "Tokyo"}}


In [50]:
#fin